# Imports

In [1]:
#Imports and installs
import transformers
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from craft_text_detector import Craft # Need to edit the saving function to prepend 0's
import torch
import os
from PIL import Image
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from tqdm import tqdm
import pandas as pd

import numpy as np
import json

import warnings

import trocr

from taxonerd import TaxoNERD

# Directories

In [2]:
# Suppressing all the huggingface warnings
SUPPRESS = True
if SUPPRESS:
    from transformers.utils import logging
    logging.set_verbosity(40)
# Turning off this warning, isn't relevant for this application
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

# Location of images
workdir = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/TROCR_Training/goodfiles/' # update this to the desired directory on scc
# Location of the segmentations
output_dir_craft = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/craft_output_files_no_detr/'
# Location to save all output files
save_dir = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data-spring2023/saved_results_trocr/'
# For ground truth labels 
workdir2 = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/scraped-data/drago_testdata/gt_labels' # update this to the desired directory on scc

# Corpus files
ALL_SPECIES_FILE = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/corpus_taxon/output/possible_species.pkl'
ALL_GENUS_FILE = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/corpus_taxon/output/possible_genus.pkl'
ALL_TAXON_FILE = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-new/ml-herbarium/corpus/corpus_taxon/corpus_taxon.txt'

# Running craft and saving the segmented images

In [5]:
# initialize the CRAFT model
craft = Craft(output_dir = output_dir_craft, 
              export_extra = False, 
              text_threshold = .7, 
              link_threshold = .4, 
              crop_type="poly", 
              low_text = .3, 
              cuda = True)

# CRAFT on images to get bounding boxes
images = []
corrupted_images = []
no_segmentations = []
boxes = {}
count= 0
img_name = []
box = []
file_types = (".jpg", ".jpeg",".png")
    
for filename in tqdm(sorted(os.listdir(workdir))):
    if filename.endswith(file_types):
        image = workdir+filename
        try:
            img = Image.open(image) 
            img.verify() # Check that the image is valid
            bounding_areas = craft.detect_text(image)
            if len(bounding_areas['boxes']): #check that a segmentation was found
                images.append(image)
                boxes[image] = bounding_areas['boxes']
                
            else:
                no_segmentations.append(image)
        except (IOError, SyntaxError) as e:
            corrupted_images.append(image)

100%|██████████| 254/254 [10:35<00:00,  2.50s/it]


# Getting all the segmented images into a dataloader, and loading model and processor for trocr

In [6]:
# Deleting empty folders, which occurs if some of the images get no segementation from CRAFT
root = output_dir_craft
folders = list(os.walk(root))[1:]
deleted = []
for folder in folders:
    if not folder[2]:
        deleted.append(folder)
        os.rmdir(folder[0])
        
# Setting up the Tr-OCR model and processor
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten") 
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten").to(device)

# Use all available gpus
model_gpu= nn.DataParallel(model,list(range(torch.cuda.device_count()))).to(device)

# Dataloader for working with gpus
trainset = datasets.ImageFolder(output_dir_craft, transform = processor)
testloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=False)

# For matching words to image
filenames = [s.replace('_crops', '') for s in list(trainset.class_to_idx)]

# For matching the image name with the label name
word_log_dic = {k: v for k,v in enumerate(filenames)}
# For matching the image name with the transriptions
words_identified = {k: [] for v,k in enumerate(filenames)}

# Saving the filenames, word_log_dic and words_identified

In [7]:
# Save filenames
with open(save_dir+'filenames.txt', 'w') as fp:
    for item in filenames:
        # write each item on a new line
        fp.write("%s\n" % item)
# Save word_log_dic 
with open(save_dir+'word_log_dic.json', 'w') as fp:
    json.dump(word_log_dic, fp)
# Save words_identified
with open(save_dir+'words_identified.json', 'w') as fp:
    json.dump(words_identified, fp)

# Running Tr-OCR on the Segmented Images from Craft

In [8]:
#Storing the outputs
results,confidence,labels = trocr.evaluate_craft_seg(model,processor, words_identified,word_log_dic,testloader,device)
#Saving all the outputs in dataframe
df = pd.DataFrame(list(zip(results,confidence,labels)),columns = ['Results','Confidence','Labels'])
df.to_pickle(save_dir+'full_results.pkl')

Transcribing Image Segments:   0%|          | 0/341 [00:00<?, ?it/s]/projectnb/sparkgrp/kabilanm/.conda/envs/trocr_env/lib/python3.9/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Transcribing Image Segments: 100%|██████████| 341/341 [05:08<00:00,  1.10it/s]


In [9]:
# First part of final csv with results, confidence level from tr-ocr, and label
combined_df = trocr.combine_by_label(df)

# Adding the image path and all bounding boxes 

df_dictionary = pd.DataFrame(boxes.items(), columns=['Image_Path', 'Bounding_Boxes'])
combined_df = pd.concat([combined_df, df_dictionary], axis=1, join='inner')
display(combined_df.head())

,Labels,Transcription,Transcription_Confidence,Image_Path,Bounding_Boxes
0,0,"[Museum d'Histoire naturelle de Paris, Herbier...","[0.2456074208021164, 0.5142542719841003, 0.997...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[3858.9062, 280.7068], [5929.0693, 357.37952..."
1,1,"[100s, 100p, 0, 100p., 2nd, 100,, top, 100, 10...","[0.005754438694566488, 0.0021388500463217497, ...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[6485.1157, 425.71875], [6612.831, 425.71875..."
2,2,"[0-, 100,, ed state., 1627083, United States n...","[0.02574569173157215, 0.04106131196022034, 0.3...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[6399.1187, 307.38126], [6524.8657, 307.3812..."
3,3,"[us, 8.810, own, copyright reserved, 1685951, ...","[0.07571566849946976, 0.05967206507921219, 0.0...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[3286.3594, 168.53125], [3539.1562, 168.5312..."
4,4,"[100, flora of the, Washington Baltimore area,...","[0.021984726190567017, 0.11620793491601944, 0....",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[6408.428, 710.46875], [6536.3125, 710.46875..."


In [10]:
#Save intermediate file
combined_df.to_pickle(save_dir+'/test.pkl')

# Reading in the ground truth files for tested images

In [11]:
# Reading in the ground truth values

gt_t = workdir2+'/taxon_gt.txt'
Taxon_truth = { line.split(":")[0] : line.split(": ")[1].strip() for line in open(gt_t) }

gt_g = workdir2+'/geography_gt.txt'
Geography_truth = { line.split(":")[0] : line.split(": ")[1].strip() for line in open(gt_g) }

gt_c = workdir2+'/collector_gt.txt'
Collector_truth = { line.split(":")[0] : line.split(": ")[1].strip() for line in open(gt_c) }

comparison_file = {"Taxon":Taxon_truth,"Countries":Geography_truth,"Collector":Collector_truth}

In [12]:
Taxon_truth_sample = {k: Taxon_truth[k] for k in list(Taxon_truth)[:10]}

# view subset of the taxon truth
print(Taxon_truth_sample)

{'1697659851': 'Euphrasia officinalis', '2573258025': 'Bryoerythrophyllum recurvirostrum', '2597666444': 'Carduus tenuiflorus', '1931288980': 'Agoseris parviflora', '1930241969': 'Spiraea canescens', '1929944910': 'Chylismia scapoidea', '1931007576': 'Carex typhina', '1928514234': 'Stachys hispida', '1928658806': 'Solanum donianum', '1931124118': 'Suaeda nigra'}


# Use TaxoNERD to recognize taxons from detected text

In [13]:
! nvidia-smi

Fri Aug 18 10:29:51 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   60C    P0              50W / 250W |   4070MiB / 16384MiB |      5%   E. Process |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:16:58_PDT_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0


In [15]:
taxonerd = TaxoNERD(prefer_gpu=False) # set to "true" if GPU is accessible

In [16]:
# utility functions for finding cosine similarity

def word2vec(word):
    from collections import Counter
    from math import sqrt

    # count the characters in word
    cw = Counter(word)
    # precomputes a set of the different characters
    sw = set(cw)
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))

    # return a tuple
    return cw, sw, lw

def cosdis(v1, v2):
    # which characters are common to the two words?
    common = v1[1].intersection(v2[1])
    # by definition of cosine distance we have
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]

In [17]:
# test "gbif_backbone" linker -> more species here
# use BERT for person and location information

taxon_output = []
confidence_output = []

nlp = taxonerd.load(
    model="en_core_eco_biobert", 
    linker="gbif_backbone", 
    threshold=0
)

/projectnb/sparkgrp/ml-herbarium-grp/summer2023/kabilanm/ml-herbarium/trocr


Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [18]:
# predict taxons for text detected from each image
for index, row in tqdm(combined_df.iterrows(), total=combined_df.shape[0]):
    temp = row["Transcription"]

    # construct a single string out of all the detected text
    input_text = " ".join(temp) 
    doc = taxonerd.find_in_text(input_text)

    try:
        # append linked taxon entity and confidence
        taxon_output.append(str(doc.entity[0][0][1]))
        confidence_output.append(float(doc.entity[0][0][2]))

    except AttributeError:
        # append empty strings when no entity is detected
        taxon_output.append("")
        confidence_output.append(float(0))

100%|██████████| 249/249 [09:30<00:00,  2.29s/it]


In [19]:
# append predicted taxon and confidence scores to the dataframe
combined_df["Taxon_Output"] = taxon_output
combined_df["Confidence_Output"] = confidence_output

In [20]:
combined_df.head()

,Labels,Transcription,Transcription_Confidence,Image_Path,Bounding_Boxes,Taxon_Output,Confidence_Output
0,0,"[Museum d'Histoire naturelle de Paris, Herbier...","[0.2456074208021164, 0.5142542719841003, 0.997...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[3858.9062, 280.7068], [5929.0693, 357.37952...",Ferraria pavonia,0.909291
1,1,"[100s, 100p, 0, 100p., 2nd, 100,, top, 100, 10...","[0.005754438694566488, 0.0021388500463217497, ...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[6485.1157, 425.71875], [6612.831, 425.71875...",Iduliella,0.415965
2,2,"[0-, 100,, ed state., 1627083, United States n...","[0.02574569173157215, 0.04106131196022034, 0.3...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[6399.1187, 307.38126], [6524.8657, 307.3812...",Clermontia persicifolia,0.593744
3,3,"[us, 8.810, own, copyright reserved, 1685951, ...","[0.07571566849946976, 0.05967206507921219, 0.0...",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[3286.3594, 168.53125], [3539.1562, 168.5312...",,0.000000
4,4,"[100, flora of the, Washington Baltimore area,...","[0.021984726190567017, 0.11620793491601944, 0....",/projectnb/sparkgrp/ml-herbarium-grp/ml-herbar...,"[[[6408.428, 710.46875], [6536.3125, 710.46875...",Elymus hystrix L.,0.791353


In [21]:
# array to store computed similarity scores
cosine_sim = []

for index, row in combined_df.iterrows():

    # extract image name from the dataframe
    img_name = row["Image_Path"].split("/")[-1][:-4]
    taxon_predicted = row["Taxon_Output"]
    taxon_gt = Taxon_truth[img_name]

    # compute cosine similarity between the predicted taxon and ground truth
    try:
        sim = cosdis(word2vec(taxon_gt), word2vec(taxon_predicted))
        cosine_sim.append(sim)
        # print(taxon_gt, taxon_predicted, sim)

    except ZeroDivisionError:
        cosine_sim.append(0)
        # print(taxon_gt, taxon_predicted,"0")

# append similarity scores to the dataframe
combined_df["Cosine_Similarity"] = cosine_sim

In [25]:
final_taxon_prediction = pd.DataFrame(columns=["Confidence_Threshold", "Taxons_Predicted", "Taxons_Accuracy_Predicted"])
temp_df = pd.DataFrame()

# generate list of similarity thresholds
# sim_threshold = [0.9]
sim_threshold =0.8

# generate list of confidence thresholds
confidence_threshold = np.arange(0, 1, 0.1)

# compute prediction accuracy at each confidence threshold
for conf_threshold in confidence_threshold:
    
    temp_df = combined_df[(combined_df["Confidence_Output"] > conf_threshold)]
    
    acc_count = (temp_df["Cosine_Similarity"] > sim_threshold).sum()

    acc_val = acc_count/len(temp_df)

    temp = [conf_threshold, acc_count, acc_val]
    final_taxon_prediction.loc[len(final_taxon_prediction)] = temp

display(final_taxon_prediction)

,Confidence_Threshold,Taxons_Predicted,Taxons_Accuracy_Predicted
0,0.0,106.0,0.477477
1,0.1,106.0,0.477477
2,0.2,106.0,0.477477
3,0.3,106.0,0.477477
4,0.4,104.0,0.495238
5,0.5,100.0,0.543478
6,0.6,90.0,0.608108
7,0.7,74.0,0.643478
8,0.8,56.0,0.746667
9,0.9,28.0,0.717949


1. We first obtain the taxon predictions with a confidence score for each taxon.
2. We then compute cosine similarities of the predicted taxons with the ground truth taxons.
3. We then, at each interval of the confidence threashold, compute number of taxons that have a high cosine similarity with the ground truth. The scores above are computed for a specific cosine similarity score ">0.8". We need to perform this step because, the taxons are matched against entries from the `ncbi_taxonomy` database (as part of TaxoNERD) and, the predicted taxon might not exactly match the ground truth and we are accounting for this using cosine similarity.

We can try to use the GBIF database to predict taxons and also experiment with different thresholds for the cosine similarity scores. But, in general, the chosen cosine similarity threshold offers an incremental performance upgrade compared to the last semester's work.